In [ ]:
# notebooks/demo_result.ipynb

import sys
import os
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# 1. Hack đường dẫn để import được code trong src
sys.path.append(os.path.abspath(os.path.join('..')))

from src import config
from src.model import MultiTaskUNet

# 2. Config & Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultiTaskUNet(encoder_name="efficientnet-b0", n_classes_seg=1, n_classes_cls=3).to(device)

# Load trọng số đã train
weights_path = os.path.join(config.WEIGHTS_DIR, "last.pth")
if os.path.exists(weights_path):
    model.load_state_dict(torch.load(weights_path, map_location=device))
    print("--> Đã load model thành công!")
else:
    print("--> Chưa có file weights!")

model.eval()

# 3. Hàm dự đoán và vẽ hình
def predict_and_plot(img_path):
    # Preprocess
    img = Image.open(img_path).convert("L")
    img_resized = img.resize(config.IMG_SIZE)
    img_np = np.array(img_resized) / 255.0
    img_tensor = torch.from_numpy(img_np).float().unsqueeze(0).unsqueeze(0).to(device)
    
    # Predict
    with torch.no_grad():
        seg_pred, cls_pred = model(img_tensor)
        
    # Xử lý kết quả
    # Mask
    mask = torch.sigmoid(seg_pred).cpu().numpy()[0, 0]
    mask = (mask > 0.5).astype(np.uint8)
    
    # Class
    probs = torch.softmax(cls_pred, dim=1).cpu().numpy()[0]
    class_names = ['AMD', 'DME', 'NORMAL']
    pred_class = class_names[np.argmax(probs)]
    confidence = np.max(probs)
    
    # Vẽ hình
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    plt.imshow(img_resized, cmap='gray')
    plt.title("Ảnh gốc (Input)")
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(mask, cmap='gray')
    plt.title("Vùng tổn thương (Segment)")
    plt.axis('off')
    
    plt.subplot(1, 3, 3)
    # Vẽ chồng mask lên ảnh gốc cho đẹp
    plt.imshow(img_resized, cmap='gray')
    plt.imshow(mask, cmap='jet', alpha=0.5) # alpha là độ trong suốt
    plt.title(f"Dự đoán: {pred_class} ({confidence*100:.1f}%)")
    plt.axis('off')
    
    plt.show()

# 4. CHẠY THỬ NGẪU NHIÊN
import random
# Lấy ảnh từ tập test (processed)
test_dir = os.path.join(config.PROCESSED_DATA_DIR, 'images', 'test')
if os.path.exists(test_dir):
    random_file = random.choice(os.listdir(test_dir))
    print(f"Đang test ảnh: {random_file}")
    predict_and_plot(os.path.join(test_dir, random_file))
else:
    print("Chưa có dữ liệu test!")